In [2]:
from keras.layers import Conv2D, UpSampling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import tensorflow as tf
from keras.losses import mean_absolute_error
from keras.models import load_model

path = "C:\\Users\\ankit\\Downloads\\humanfaces"
train_datagen = ImageDataGenerator(rescale=1. / 255)
train = train_datagen.flow_from_directory(path, 
                                          target_size=(256, 256), 
                                          batch_size=1000, 
                                          class_mode=None)

Found 6981 images belonging to 1 classes.


In [3]:
#Encoder
model = Sequential()
model.add(Conv2D(16, (3,3), activation='relu', padding='same', strides=2, input_shape=(256, 256, 1)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(Conv2D(64, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(Conv2D(128, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))

In [4]:
#Decoder
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(Conv2D(2, (3,3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=mean_absolute_error, metrics=['accuracy'])

In [31]:
for i in range(len(train)):
    X =[]
    Y =[]
    for img in train[i]:
      try:
          lab = rgb2lab(img)
          X.append(lab[:,:,0]) 
          Y.append(lab[:,:,1:] / 128)
      except:
         print('error')
    X = np.array(X)
    Y = np.array(Y)
    X = X.reshape(X.shape+(1,))
    print(X.shape)
    print(Y.shape)
    print("batch: ", i+1)
    model.fit(X,Y,validation_split=0.1, epochs=3, batch_size=20)

(1000, 256, 256, 1)
(1000, 256, 256, 2)
batch:  1
Epoch 1/3
45/45 [==============================] - 102s 2s/step - loss: 0.0375 - accuracy: 0.6198 - val_loss: 0.0411 - val_accuracy: 0.5606
Epoch 2/3
45/45 [==============================] - 102s 2s/step - loss: 0.0354 - accuracy: 0.6248 - val_loss: 0.0409 - val_accuracy: 0.5626
Epoch 3/3
45/45 [==============================] - 97s 2s/step - loss: 0.0344 - accuracy: 0.6225 - val_loss: 0.0402 - val_accuracy: 0.5617
(1000, 256, 256, 1)
(1000, 256, 256, 2)
batch:  2
Epoch 1/3
45/45 [==============================] - 103s 2s/step - loss: 0.0370 - accuracy: 0.6069 - val_loss: 0.0389 - val_accuracy: 0.5945
Epoch 2/3
45/45 [==============================] - 98s 2s/step - loss: 0.0357 - accuracy: 0.6178 - val_loss: 0.0392 - val_accuracy: 0.6379
Epoch 3/3
45/45 [==============================] - 93s 2s/step - loss: 0.0344 - accuracy: 0.6186 - val_loss: 0.0385 - val_accuracy: 0.6432
(1000, 256, 256, 1)
(1000, 256, 256, 2)
batch:  3
Epoch 1/3
45/

In [35]:
model.save("Rango.h5")

In [3]:
model = load_model('Rango.h5')

In [4]:
img1_color=[]

img1=img_to_array(load_img("C:\\Users\\ankit\\Downloads\\brain.jpeg"))
img1 = resize(img1 ,(256, 256))
img1_color.append(img1)

img1_color = np.array(img1_color, dtype=float)
img1_color = rgb2lab(1.0/255*img1_color)[:,:,:,0]
img1_color = img1_color.reshape(img1_color.shape+(1,))

output1 = model.predict(img1_color)
output1 = output1*160

result = np.zeros((256, 256, 3))
result[:,:,0] = img1_color[0][:,:,0]
result[:,:,1:] = output1[0]
imsave("C:\\Users\\ankit\\Downloads\\braincol.jpg", lab2rgb(result))

1/1 [==============================] - 1s 620ms/step


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
